# Simulaciones para generar un dataset de hormigoneras

En este documento, generaremos sinteticamente un dataset que refleje la produccion de una hormigonera.

In [40]:
import pandas as pd
import numpy as np
import random
import datetime
from datetime import timedelta
from faker import Faker

In [41]:
drivers = pd.DataFrame()
fake = Faker()

In [42]:
start_date = datetime.date(2015,1,1)
end_date = start_date + timedelta(days=(365*9))
dates = pd.bdate_range(start_date, end_date)

# Choferes

In [43]:
def generate_random_date(start_date):
    max_days = 365 * 9  # 5 años
    random_number_of_days = random.randint(0, max_days)
    return start_date + timedelta(days=random_number_of_days)

In [44]:
drivers["chofer_id"] = [i for i in range(1,50)]
drivers["camion"] = [i for i in range(25,74)]
drivers["edad"] = [random.randint(25,55) for i in range(1,50)]
drivers['fecha_ingreso'] = drivers['chofer_id'].apply(lambda x: generate_random_date(start_date))
drivers['fecha_ingreso'] = pd.to_datetime(drivers['fecha_ingreso'])
drivers["Nombre"] = [fake.name() for i in range(1,50)]

drivers.to_csv('dim_choferes.csv')

In [45]:
drivers.head()

,chofer_id,camion,edad,fecha_ingreso,Nombre
0,1,25,25,2016-02-24,Lindsay Reed
1,2,26,42,2020-05-31,Anthony Rivera
2,3,27,28,2015-09-01,Tamara Sullivan
3,4,28,46,2016-02-05,Eddie Odom
4,5,29,29,2016-05-02,Jason Perkins


# Clientes

In [46]:
def generar_dataframe_clientes(cantidad):
    fake = Faker()
    datos_clientes = []

    for _ in range(cantidad):
        nombre_empresa = fake.company()
        direccion = fake.address()
        telefono = fake.phone_number()
        email = fake.company_email()

        cliente = {
            "Nombre Empresa": nombre_empresa,
            "Dirección": direccion,
            "Teléfono": telefono,
            "Email": email
        }

        datos_clientes.append(cliente)

    # Crear un DataFrame de pandas
    df_clientes = pd.DataFrame(datos_clientes)
    return df_clientes

if __name__ == "__main__":
    cantidad_clientes = 500
    dataframe_clientes = generar_dataframe_clientes(cantidad_clientes)


dataframe_clientes["cliente_id"] = [i for i in range(1,len(dataframe_clientes)+1)]

dataframe_clientes.to_csv('dim_clientes.csv')

In [47]:
dataframe_clientes.head()

,Nombre Empresa,Dirección,Teléfono,Email,cliente_id
0,Lopez-Houston,"13641 Mark Neck Apt. 656\nJenniferton, MN 82105",001-490-658-4068x170,aperez@hubbard.com,1
1,Sparks-Adams,0881 Williams Expressway Suite 392\nEast Alexa...,(077)447-7161,wramsey@lopez-torres.com,2
2,"Morgan, Lewis and Anderson","9731 Jennings Way\nNew Chelseaton, MA 59688",+1-653-646-4750,avillarreal@white.com,3
3,Wright Ltd,"595 Melissa Greens Suite 402\nNew Sandra, WA 4...",(387)839-8171x2780,jonesnancy@khan-dawson.com,4
4,Davis PLC,"16338 Julian Stream Suite 311\nPort Isaiah, CO...",608.112.7496x52917,boydjason@hanson.info,5


# Formulas

In [48]:
dataframe_formulas = pd.read_excel('formulas_simulacion_hormigoneras.xlsx', sheet_name='formulas')
dataframe_formulas.head()

dataframe_formulas.to_csv('dim_formulas.csv')

In [49]:
dataframe_formulas.head()

,formula_id,Especificada,asentamiento,TMN,cemento,agua,arena,piedra_BI,piedra_6_19,piedra_20_30,aditivo,chances
0,1,21,8,19,309.090909,170,950,0.0,970.909091,0.000000,24.727273,0.10
1,2,21,12,19,318.181818,175,950,0.0,956.818182,0.000000,25.454545,0.30
2,3,21,15,19,327.272727,180,950,0.0,942.727273,0.000000,26.181818,0.10
3,4,21,18,19,336.363636,185,950,0.0,928.636364,0.000000,26.909091,0.05
4,5,21,8,25,309.090909,170,980,0.0,470.454545,470.454545,24.727273,0.15


# Registro de viajes diarios

In [50]:
remito = 1000
minutos_diarios_limite = 11 * 60  # 11 horas por día 660 480
minutos_diarios = 0

# Columnas para el DataFrame de viajes
columnas_viajes = ['chofer_id', 'remito', 'fecha', 'hora_inicio_carga', 'hora_fin_carga', 'cliente_id', 'formula', 'cantidad']

# Lista para almacenar los datos de los viajes
datos_viajes = []

# Inicializar el contador de minutos
contador_minutos = 0

# Bucle principal para simular los viajes
for date in dates:
    for _, chofer_row in drivers.iterrows():

        chofer_id = chofer_row['chofer_id']
        fecha_ingreso = chofer_row['fecha_ingreso']
        if date < fecha_ingreso:
            continue
        
        horas_trabajadas = max(0, np.random.normal(10, 2))
        minutos_trabajados = horas_trabajadas * 60
        # Restablecer el contador de minutos al inicio de cada día
        contador_minutos = 0

        while contador_minutos < minutos_trabajados: # controla que la hora del chofer este entre 8 y 11 horas diarias
            # Calcular la duración del viaje
            duracion_viaje = np.random.normal(loc=180, scale=60) * random.uniform(0.8,1.2)  # 2 horas de media, 40 minutos de desviación estándar

            # Calcular las horas de inicio y fin de carga
            hora_inicio_carga = datetime.datetime(date.year, date.month, date.day, 8, 0) + timedelta(minutes=contador_minutos)
            hora_fin_carga = hora_inicio_carga + timedelta(minutes=duracion_viaje)

            # Seleccionar un cliente al azar
            cliente_id = random.choice(dataframe_clientes["cliente_id"])
            # Seleccionar una formula
            probabilidades_formula = dataframe_formulas["chances"]
            formula = random.choices(dataframe_formulas["formula_id"], weights= probabilidades_formula, k=1)[0]

            # Seleccionar una cantidad
            cantidad = random.randint(6,8)

            # Agregar datos del viaje a la lista
            datos_viaje = [chofer_id, remito, date, hora_inicio_carga, hora_fin_carga, cliente_id, formula, cantidad]
            datos_viajes.append(datos_viaje)

            # Incrementar el contador de minutos
            contador_minutos += duracion_viaje + 10  # 10 minutos de margen entre viajes

            # Incrementar el número de remito
            remito += 1

# Crear un DataFrame de Pandas con los datos de los viajes
df_viajes = pd.DataFrame(datos_viajes, columns=columnas_viajes)

df_viajes.to_csv('fact_viajes.csv')


# Reclamos

In [51]:
chance_de_reclamo = 0.05 # chance que de el viaje contenga un reclamo

# establezco algunos posibles reclamos que podrian suceder
tipos_de_reclamos = ['volumen', 'fisuracion', 'resistencia', 'demoras', 'fragüe', 'asentamiento', 'formula']

# definimos una probabilidad para cada uno de los reclamos, entendiendo que algunos son mas frecuentes que otros
probabilidades_reclamos = [0.1, 0.3, 0.1, 0.15, 0.05, 0.3, 0.05]

remitos = df_viajes["remito"]

# Lista para almacenar datos de reclamos
datos_reclamos = []

for remito in remitos:
    # Verificar si este viaje tiene un reclamo
    if random.random() < chance_de_reclamo:
        # Si hay reclamo, agregar datos a la lista
        tipo_reclamo = random.choices(tipos_de_reclamos, weights=probabilidades_reclamos, k=1)[0]
        datos_reclamo = {"remito": remito, "tipo": tipo_reclamo}
        datos_reclamos.append(datos_reclamo)

# Crear un DataFrame de Pandas con los datos de reclamos
df_reclamos = pd.DataFrame(datos_reclamos)

df_reclamos.to_csv('fact_reclamos.csv')

In [52]:
df_reclamos.head()

,remito,tipo
0,1009,fisuracion
1,1011,volumen
2,1019,fisuracion
3,1028,fisuracion
4,1037,resistencia


# Calidad

In [53]:
chance_de_control = 0.15 # chance que de el viaje reciba un control de calidad

# Lista para almacenar datos de controles
datos_controles = []

for remito in remitos:
    # Verificar si este viaje tiene un control de calidad
    if random.random() < chance_de_control:
        # Si hay control, agregar datos a la lista
        # Qué formula fue muestreada
        formula = df_viajes['formula'].loc[df_viajes['remito'] == remito].iloc[0]
        
        # resistencia especificada de la formula muestreada
        especificada = dataframe_formulas['Especificada'].loc[dataframe_formulas['formula_id'] == formula].iloc[0]        
        
        # resultado del muestreo
        resistencia_28 = np.random.normal(loc=(especificada+1.282*3), scale=3) * random.uniform(0.95,1.05)
        resistencia_7 = resistencia_28 * np.random.uniform(0.6, 0.7)
        
        datos_control = {"remito": remito, "formula": formula, "edad": 28, 'especificada': especificada, 'resistencia_7': resistencia_7, 'resistencia_28': resistencia_28}
        datos_controles.append(datos_control)

# Crear un DataFrame de Pandas con los datos de reclamos
df_controles = pd.DataFrame(datos_controles)

df_controles.to_csv('fact_calidad.csv')

# Volumen

In [54]:
def simulate_hormigon_volumen(fecha, variaciones_estacionales, impacto_economico, probabilidad_proyecto_grande):
    base_volumen = 100  # Volumen base diario

    # Ajuste estacional
    if fecha.month in [3, 4, 5, 6, 7, 8]:  # Meses de primavera/verano
        estacional = variaciones_estacionales['alto']
    else:
        estacional = variaciones_estacionales['bajo']
    
    # Ajuste económico
    economico = impacto_economico[fecha.year]  # Impacto económico por año
    
    # Proyecto de gran escala
    if random.random() < probabilidad_proyecto_grande:
        proyecto_grande = np.random.choice([0, 1], p=[0.9, 0.1])  # 10% de probabilidad de un gran proyecto
    else:
        proyecto_grande = 0
    
    # Interrupciones operativas
    interrupcion = np.random.choice([1, 0], p=[0.95, 0.05])  # 5% de probabilidad de interrupción
    
    # Cálculo final del volumen
    volumen = base_volumen * estacional * economico * (1 + proyecto_grande) * interrupcion
    return volumen
